In [68]:
import pandas as pd

In [69]:
file_path ='mobile_addiction.csv'
df=pd.read_csv(file_path)

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13589 entries, 0 to 13588
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          13589 non-null  int64 
 1   daily_screen_time   13589 non-null  int64 
 2   app_sessions        13589 non-null  int64 
 3   social_media_usage  13589 non-null  int64 
 4   gaming_time         13589 non-null  int64 
 5   notifications       13589 non-null  int64 
 6   night_usage         13589 non-null  int64 
 7   age                 13589 non-null  int64 
 8   work_study_hours    13589 non-null  int64 
 9   stress_level        13589 non-null  int64 
 10  apps_installed      13589 non-null  int64 
 11  addicted            13589 non-null  object
dtypes: int64(11), object(1)
memory usage: 1.2+ MB


In [71]:
df.head()

,Unnamed: 0,daily_screen_time,app_sessions,social_media_usage,gaming_time,notifications,night_usage,age,work_study_hours,stress_level,apps_installed,addicted
0,0,2,29,0,0,49,0,44,5,3,35,not addicted
1,1,6,29,1,2,65,1,29,5,9,21,addicted
2,2,9,28,2,0,57,3,28,7,5,39,addicted
3,3,6,39,2,0,69,1,28,6,8,24,addicted
4,4,5,37,3,1,64,2,27,4,5,26,addicted


In [72]:
df['addicted'] = df['addicted'].map({
    'not addicted': 1,
    'addicted':     0
}).astype(int)

df = df.drop(columns=['Unnamed: 0'])

df.describe()

,daily_screen_time,app_sessions,social_media_usage,gaming_time,notifications,night_usage,age,work_study_hours,stress_level,apps_installed,addicted
count,13589.000000,13589.000000,13589.000000,13589.000000,13589.000000,13589.000000,13589.000000,13589.00000,13589.000000,13589.000000,13589.000000
mean,3.768121,30.037604,1.545956,1.034219,60.015306,0.990213,33.055707,5.97807,4.265288,27.532048,0.496210
std,1.897280,7.406585,1.204830,0.993172,12.732064,0.946606,10.118145,2.07139,2.288454,5.894790,0.500004
min,0.000000,8.000000,0.000000,0.000000,25.000000,0.000000,15.000000,0.00000,0.000000,10.000000,0.000000
25%,2.000000,25.000000,1.000000,0.000000,50.000000,0.000000,25.000000,5.00000,3.000000,23.000000,0.000000
50%,4.000000,30.000000,1.000000,1.000000,59.000000,1.000000,33.000000,6.00000,4.000000,27.000000,0.000000
75%,5.000000,35.000000,2.000000,2.000000,70.000000,2.000000,41.000000,7.00000,6.000000,31.000000,1.000000
max,12.000000,60.000000,7.000000,6.000000,103.000000,6.000000,55.000000,14.00000,10.000000,55.000000,1.000000


# Clustering k-means

In [73]:
X = df.drop(columns=['addicted'])
y = df['addicted']

In [ ]:
import numpy as np

def kmeans(X, k=2, max_iters=100, tol=1e-6, random_state=None):
    rng = np.random.default_rng(random_state) # A tool to calculate random variable
    initial_idx = rng.choice(X.shape[0], size=k, replace=False) # Initialize centroids by sampling k distinct points in a X.shape[0] dimensional space 
    centroids = X[initial_idx] # list of centroids
    
    for i in range(max_iters):
        distances = np.linalg.norm(X[:, None, :] - centroids[None, :, :], axis=2) # all the distances between each centroid and all the points
        labels = np.argmin(distances, axis=1) # take the min of that, so we have a label for each point with the centroid it is closest to
        
        # Compute new centroids
        new_centroids = np.zeros_like(centroids)
        for j in range(k):
            points_in_j = X[labels == j] # list of points that is closetest to k_j 
            if len(points_in_j) > 0:
                new_centroids[j] = points_in_j.mean(axis=0)
            else:
                # If a cluster lost all its points, leave it unchanged, this is done to keep the algo stable, and this will keep the empty cluster in its place, not moving it
                new_centroids[j] = centroids[j]
    
    
        shifts = np.linalg.norm(new_centroids - centroids, axis=1) # how much change happened to clusters, so does the code converge or not?
        if np.all(shifts < tol):
            break
        
        centroids = new_centroids
    
    return centroids, labels

X_np = X.values
C, lbls = kmeans(X_np, k=2, random_state=42)
print("Final centroids:\n", C)
print(lbls)


Final centroids:
 [[ 4 34  1  1 70  1 28  5  5 29]
 [ 3 25  1  0 49  0 37  6  3 25]]
[1 0 0 ... 0 1 1]


In [78]:
y_np = y.values

correct = np.sum(lbls == y_np)
total   = len(y_np)
accuracy = correct / total
print(f"{correct} out of {total} correct → {accuracy*100:.2f}%")

12852 out of 13589 correct → 94.58%
